In [1]:
from ultralytics import YOLO
import cv2
import math
from tqdm import tqdm
import os
import moviepy.editor as mpe

In [2]:
def retrieve_audio(video):
    video_clip = mpe.VideoFileClip(video)

    # Extract the audio from the video clip
    audio_clip = video_clip.audio
    video_clip.close()

    return audio_clip

In [3]:
def combine_audio(video, audio, outname, fps):
    final_clip = video.set_audio(audio)
    # os.remove(outname)
    print(type(final_clip))
    final_clip.write_videofile(f"f{outname}", fps=fps)

In [4]:
video_path = "crowded_space.mp4"
output_path = "video.mp4"

In [5]:
model = YOLO('yolov8n-face.pt')

In [6]:
# audio = retrieve_audio(video_path)

In [7]:
vidcap = cv2.VideoCapture(video_path)
success, img = vidcap.read()

In [8]:
frame_number = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
frame_size = (int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
              int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video=cv2.VideoWriter(filename=output_path, 
                      fourcc=fourcc, 
                      fps=fps, 
                      frameSize=frame_size)

In [10]:
def blur_box(frame, box, min_conf=0.2):
    if math.ceil((box.conf[0]*100))/100 > min_conf:
        x1, y1, x2, y2 = box.xyxy[0]
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        h, w = y2-y1, x2-x1

        ROI = frame[y1:y1+h, x1:x1+w]
        blur = cv2.GaussianBlur(ROI, (51,51), 0) 
        frame[y1:y1+h, x1:x1+w] = blur
    return frame


In [11]:
for i in tqdm(range(frame_number)):
    results = model(img, stream=True, verbose=False)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            img = blur_box(img, box)

    video.write(img)
    success, img = vidcap.read()

vidcap.release()
video.release()

100%|██████████| 460/460 [00:39<00:00, 11.70it/s]


In [12]:
type(audio)

NameError: name 'audio' is not defined

In [ ]:
from moviepy.editor import *

# Define the paths of the audio and video files
video_path = output_path

# Create instances of VideoFileClip and AudioFileClip
video_clip = VideoFileClip(video_path)

# Merge the audio and video clips
video_clip_with_audio = video_clip.set_audio(audio)

# Write the merged video file to a new file
# video_clip_with_audio.write_videofile("path/to/output/file.mp4")